In [1]:
# Install if not already installed
# !pip install mne numpy

import mne
import numpy as np


In [2]:
# Replace with the path to your EEG file
eeg_file_path = r"C:\Users\Ferran\Desktop\TFM\TFM_EEG\DATASET\MI\SUB_2\ICA\L\S001\sub-S001_L_eeg_raw.fif"

# Load EEG data
raw = mne.io.read_raw_fif(eeg_file_path, preload=True)
print(raw)  # Print basic information about the data

# Display the sampling rate and data duration
sampling_rate = raw.info['sfreq']
duration = raw.n_times / sampling_rate
print(f"Sampling Rate: {sampling_rate} Hz")
print(f"Data Duration: {duration} seconds")


Opening raw data file C:\Users\Ferran\Desktop\TFM\TFM_EEG\DATASET\MI\SUB_2\ICA\L\S001\sub-S001_L_eeg_raw.fif...
    Range : 0 ... 46975 =      0.000 ...   183.507 secs
Ready.
Reading 0 ... 46975  =      0.000 ...   183.507 secs...
<Raw | sub-S001_L_eeg_raw.fif, 10 x 46976 (183.5 s), ~3.6 MB, data loaded>
Sampling Rate: 255.9844207763672 Hz
Data Duration: 183.51116781844752 seconds


In [6]:
# Replace with the path to your event data file
event_file_path = r"C:\Users\Ferran\Desktop\TFM\TFM_EEG\DATASET\MI\SUB_2\ICA\L\S001\sub-S001_L_event_data.npz"

# Load event data
event_data = np.load(event_file_path)
print(event_data.files)  # Display keys to check what data is stored

# Extract event times (assuming 'event_times' is the key for times in the .npz)
event_times = event_data['event_timestamps']  # Adjust if the key is different

# Convert event times to seconds if necessary
event_times_sec = event_times / sampling_rate

print("Event Times (in seconds):", event_times_sec)


['event_timestamps', 'data_segments']
Event Times (in seconds): [60056.88628932 60056.90782833 60056.92939273 60056.95094274
 60056.97248619 60056.99403929 60057.01558735 60057.03714486
 60057.05870241 60057.08020559 60057.10178829 60057.12340048
 60057.14489042 60057.16645217 60057.18799177 60057.20947581
 60057.23102888 60057.25258716 60057.27413159 60057.29568017
 60057.31722883 60057.33878517 60057.36040447 60057.38195098
 60057.40349928 60057.42505293 60057.44660302 60057.46815729
 60057.48969847 60057.5111833 ]


In [7]:
# Identify events that are out of bounds
out_of_bounds_events = event_times_sec[(event_times_sec < 0) | (event_times_sec > duration)]

if len(out_of_bounds_events) > 0:
    print("Out of bounds events found:")
    print(out_of_bounds_events)
else:
    print("All events are within the EEG data duration.")


Out of bounds events found:
[60056.88628932 60056.90782833 60056.92939273 60056.95094274
 60056.97248619 60056.99403929 60057.01558735 60057.03714486
 60057.05870241 60057.08020559 60057.10178829 60057.12340048
 60057.14489042 60057.16645217 60057.18799177 60057.20947581
 60057.23102888 60057.25258716 60057.27413159 60057.29568017
 60057.31722883 60057.33878517 60057.36040447 60057.38195098
 60057.40349928 60057.42505293 60057.44660302 60057.46815729
 60057.48969847 60057.5111833 ]


In [12]:
# Option 3: If timestamps need an offset adjustment, subtract the minimum timestamp
event_times_sec = event_times - event_times.min()
print("Adjusted Event Times (in seconds):", event_times_sec)

# Check for out-of-bounds events
out_of_bounds_events = event_times_sec[(event_times_sec < 0) | (event_times_sec > duration)]
if len(out_of_bounds_events) > 0:
    print("Out of bounds events found:")
    print(out_of_bounds_events)
else:
    print("All events are within the EEG data duration.")


Adjusted Event Times (in seconds): [  0.           5.51365075  11.0337993   16.55026634  22.06505479
  27.58231324  33.09828029  38.61666694  44.13506258  49.63954403
  55.16437738  60.69676162  66.19785137  71.71732292  77.23112537
  82.73070422  88.24795537  93.76653882  99.28157796 104.79767791
 110.31379946 115.83188631 121.36609145 126.8816628  132.39769005
 137.9150892  143.43157635 148.94913459 154.46333964 159.96312159]
All events are within the EEG data duration.


In [13]:
# Create an MNE events array if events are within bounds (needed for plotting)
# Assuming event markers are simply '1' and no other info is required, adjust if necessary
events_array = np.array([[int(time * sampling_rate), 0, 1] for time in event_times_sec])

# Plot EEG data with events
raw.plot(events=events_array, title="EEG Data with Events")
